In [1]:
import scanpy as sc
import anndata
from anndata import AnnData
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from collections import Counter
import seaborn as sns
sc.settings.verbosity = 0
sc.settings.set_figure_params(dpi=150, color_map='viridis')
sc.logging.print_versions()

/home/jih8/anaconda3/lib/python3.7/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


scanpy==1.4.3 anndata==0.6.22.post1 umap==0.3.9 numpy==1.16.2 scipy==1.2.1 pandas==1.1.2 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
pwd

'/pylon5/mc5plqp/jih8/Peanut_b2/Peanut_Fiona/My_pipline/DataIntegration/SoupX'

In [4]:
raw_adata = anndata.read_h5ad('/pylon5/mc5plqp/jih8/Peanut_b2/Peanut_Fiona/My_pipline/raw_adata_afterDB_beforePre_SoupX.h5ad')
raw_adata

AnnData object with n_obs × n_vars = 80902 × 53573 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8'

In [5]:
#give another colum 
sample_ids = list(map(int,raw_adata.obs['batch']))
monkey_names = ['Penut','Penut','Penut','Penut','Penut','Penut','Fiona','Fiona','Fiona']
monkey_names = pd.Series([monkey_names[sample_id] for sample_id in sample_ids])
raw_adata.obs['tech'] = list(monkey_names)
raw_adata

AnnData object with n_obs × n_vars = 80902 × 53573 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet', 'tech'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8'

In [7]:
#give another colum 
sample_ids = list(map(int,raw_adata.obs['batch']))
region_names = ['caudate','nacc','putamen','caudate','nacc','putamen','nacc_Fiona','caudate_Fiona','putamen_Fiona']
region_names = pd.Series([region_names[sample_id] for sample_id in sample_ids])
raw_adata.obs['regions_1'] = list(region_names)
raw_adata

AnnData object with n_obs × n_vars = 80902 × 53573 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet', 'tech', 'regions_1'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8'

In [8]:
#give another colum 
sample_ids = list(map(int,raw_adata.obs['batch']))
region_names = ['caudate','nacc','putamen','caudate','nacc','putamen','nacc','caudate','putamen']
region_names = pd.Series([region_names[sample_id] for sample_id in sample_ids])
raw_adata.obs['regions_2'] = list(region_names)
raw_adata

AnnData object with n_obs × n_vars = 80902 × 53573 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet', 'tech', 'regions_1', 'regions_2'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8'

In [9]:
sc.pp.filter_cells(raw_adata, min_genes=200)

/home/jih8/anaconda3/lib/python3.7/site-packages/anndata/core/anndata.py:1239: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(df_full[k]):


In [10]:
sc.pp.filter_genes(raw_adata, min_cells=5)

In [11]:
sc.pp.filter_genes(raw_adata, min_counts=15)

In [12]:
mito_genes = raw_adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
raw_adata.obs['percent.mt'] = np.sum(raw_adata[:, mito_genes].X, axis=1).A1 / np.sum(raw_adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
raw_adata.obs['nCount_RNA'] = raw_adata.X.sum(axis=1).A1

In [13]:
raw_adata

AnnData object with n_obs × n_vars = 80902 × 31657 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet', 'tech', 'regions_1', 'regions_2', 'n_genes', 'percent.mt', 'nCount_RNA'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8', 'n_cells', 'n_counts'

In [14]:
raw_adata.var['n_counts']

index
CICP27           23.378494
AP006222.1       62.885563
MTND1P23         37.702255
MTND2P28      12118.856445
MTCO1P12       2463.187012
                  ...     
RPL18AP2        116.552849
DIP2A         46272.921875
S100B         16044.850586
PRMT2         34363.785156
DSTNP1           16.475237
Name: n_counts, Length: 31657, dtype: float32

In [15]:
raw_adata.var['n_cells']

index
CICP27           24
AP006222.1       68
MTND1P23         55
MTND2P28      12047
MTCO1P12       3717
              ...  
RPL18AP2        124
DIP2A         30746
S100B         17153
PRMT2         29540
DSTNP1           17
Name: n_cells, Length: 31657, dtype: int64

In [16]:
raw_adata.obs['nCount_RNA']

index
AAACCCAAGAGCGACT-1-0    9448.478516
AAACCCACAAATACAG-1-0    4037.190918
AAACCCACAAGGTTGG-1-0    2464.473145
AAACCCACACGTTCGG-1-0     824.900513
AAACCCACACTGGATT-1-0    5321.896973
                           ...     
TTTGTTGCATATTCGG-1-8    1540.000000
TTTGTTGCATTCTGTT-1-8    2636.000000
TTTGTTGGTCCCGTGA-1-8    1434.000000
TTTGTTGGTCGCTGCA-1-8    3004.000000
TTTGTTGGTCGTACAT-1-8    3567.000000
Name: nCount_RNA, Length: 80902, dtype: float32

In [17]:
ribo_genes = raw_adata.var_names.str.startswith(("RPS","RPL"))
print(sum(ribo_genes))
raw_adata.obs['percent_ribo'] = np.sum(raw_adata[:, ribo_genes].X, axis=1).A1 / np.sum(raw_adata.X, axis=1).A1

399


In [18]:
raw_adata = raw_adata[:,~ribo_genes]

In [19]:
raw_adata

View of AnnData object with n_obs × n_vars = 80902 × 31258 
    obs: 'batch', 'cell_id', 'sample_names', 'regions', 'doublet', 'tech', 'regions_1', 'regions_2', 'n_genes', 'percent.mt', 'nCount_RNA', 'percent_ribo'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-2', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'feature_types-6', 'gene_ids-7', 'feature_types-7', 'gene_ids-8', 'feature_types-8', 'n_cells', 'n_counts'

In [20]:
#AnnData.write(raw_adata, filename='/pylon5/mc5plqp/jih8/Peanut_b2/Peanut_Fiona/My_pipline/DataIntegration/SoupX/raw_adata_afterDB_SoupX_colomns_added.h5ad', compression=None, compression_opts=None, force_dense=None)

/home/jih8/anaconda3/lib/python3.7/site-packages/anndata/core/anndata.py:1328: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  and not is_categorical(df[key])
Trying to set attribute `.obs` of view, making a copy.
... storing 'tech' as categorical
Trying to set attribute `.obs` of view, making a copy.
... storing 'regions_1' as categorical
Trying to set attribute `.obs` of view, making a copy.
... storing 'regions_2' as categorical
/home/jih8/anaconda3/lib/python3.7/site-packages/anndata/core/anndata.py:108: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif is_categorical(df[k]):
/home/jih8/anaconda3/lib/python3.7/site-packages/anndata/core/anndata.py:101: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[k]) and not is_categorical(df[k]):
